In [ ]:
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
import numpy as np
from keras.layers import Input
from keras.models import Model
import tensorflow as tf
import scipy
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
import scipy.misc

from models import *
import random
from keras import backend as K
sess = tf.Session()
K.set_session(sess)

In [ ]:
class Batch():
    def __init__(self, dirname):
        all_files = self.get_files(dirname)

        self.num_all = len(all_files)
        self.num_train = int(self.num_all*0.80)
        self.num_test = self.num_all - self.num_train

        random.shuffle(all_files)

        self.train_files = all_files[:self.num_train]
        self.test_files = all_files[self.num_train:]
        self.train_start = 0
        self.test_start = 0
        
    def get_files(self, dirname, mode='i'):
        img_dir = dirname + '/images/'
        contour_dir = dirname + '/contours_' + mode + '/'
        img_names = [join(img_dir, f) for f in listdir(img_dir) if isfile(join(img_dir, f))]
        contour_names = [join(contour_dir, f) for f in listdir(contour_dir) 
                        if isfile(join(contour_dir, f))]
        img_names.sort()
        contour_names.sort()
        return list(zip(img_names, contour_names))

    def image_augment(self, img1, img2):
        rand_num = np.random.rand()
        if rand_num < 0.50:
            img1 = np.fliplr(img1)
            img2 = np.fliplr(img2)
        else:
            return img1, img2
        return img1, img2

    def read_images(self, files):
        batch_size = len(files)
        imgs = np.ones((batch_size, 256, 256, 1))
        contours = np.ones((batch_size, 256, 256, 1))
        img_contours = np.ones((batch_size, 256, 256, 2))
        for i in range(len(files)):
            img = scipy.misc.imread(files[i][0])
            contour = scipy.misc.imread(files[i][1])
            img, contour = self.image_augment(img, contour)
            img = scipy.misc.imresize(img, (256, 256))/255.0
            contour = scipy.misc.imresize(contour, (256, 256))/255.0
            imgs[i,:,:,:] = np.expand_dims(img, -1)#.transpose(2,0,1)
            contours[i,:,:,:] = np.expand_dims(contour, -1)
            img_contours[i,:,:,:] = np.stack([contour, img], axis=-1)
        return imgs, contours, img_contours


    def get_next_batch(self, batch_size):
        is_running = True
        end = min(self.train_start + batch_size, self.num_train)
        files = self.train_files[self.train_start:end]
        self.train_start += batch_size
        if end == self.num_train:
            self.train_start = 0
            random.shuffle(self.train_files)
            is_running = False
        imgs, contours, img_contours = self.read_images(files)
        return imgs, contours, img_contours, is_running
    
    def get_next_testbatch(self, batch_size):
        is_running = True
        end = min(self.test_start + batch_size, self.num_test)
        files = self.test_files[self.test_start:end]
        self.test_start += batch_size
        if end == self.num_test:
            self.test_start = 0
            is_running = False
        imgs, contours, img_contours = self.read_images(files)
        return imgs, contours, img_contours, is_running
    

In [ ]:
dirname = './Train_Set'

batch = Batch(dirname)

In [ ]:
img_size = 256
trainable = True
lr = 5e-5
c = 1e-2

seg_inputs_pl = tf.placeholder(tf.float32, shape=[None, img_size, img_size, 1])
d_inputs_pl = tf.placeholder(tf.float32, shape=[None, img_size, img_size, 2])

seg_input = Input(shape=(img_size, img_size, 1))
seg_output = unet(seg_input, trainable=trainable)

d_input = Input(shape=(img_size, img_size, 2))
d_out = discriminator(d_input, trainable=trainable)

seg_model = Model(inputs=seg_input, outputs=seg_output)
d_model = Model(inputs=d_input, outputs=d_out)

seg_weights = [w for w in tf.global_variables() if 'unet' in w.name]
d_weights = [w for w in tf.global_variables() if 'discriminator' in w.name]

pred_mask = seg_model(seg_inputs_pl)
fake_inputs = tf.concat([pred_mask, seg_inputs_pl], axis=-1)
d_real = d_model(d_inputs_pl)
d_fake = d_model(fake_inputs)

loss_seg = tf.reduce_mean(d_fake)
loss_d = tf.reduce_mean(d_real) - tf.reduce_mean(d_fake)

opt_seg = tf.train.RMSPropOptimizer(lr)
opt_d = tf.train.RMSPropOptimizer(lr)

grad_seg = opt_seg.compute_gradients(loss_seg, seg_weights)
grad_d = opt_d.compute_gradients(loss_d, d_weights)

train_op_seg = opt_seg.apply_gradients(grad_seg)
train_op_d = opt_d.apply_gradients(grad_d)

clip_d_weights = [w.assign(tf.clip_by_value(w, -c, c)) for w in d_weights]

init = tf.global_variables_initializer()

tf.summary.scalar("Wasserstien_Distance", loss_d)

tf.summary.image("Generated_Mask", pred_mask, max_outputs=1)

tf.summary.image("Real_Mask", tf.expand_dims(d_inputs_pl[:,:,:,0], -1), max_outputs=1)

merged_summary_op = tf.summary.merge_all()

In [ ]:
num_epochs = 1
logs_path = './logs'

saver = tf.train.Saver()

batch_size = 5
ep_critic = 5
with sess.as_default():
    sess.run(init)
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    for epoch in range(num_epochs):
        count = 0
        is_running = True
        while is_running:
            if count%100 == 0:
                ep_critic = 1
            else:
                ep_critic = 1
            for i in range(ep_critic):
                imgs, _, img_contours, is_running = batch.get_next_batch(batch_size)
                sess.run([train_op_d], 
                         feed_dict={seg_inputs_pl: imgs, d_inputs_pl:img_contours,
                                    K.learning_phase(): 1})
                sess.run([clip_d_weights], 
                         feed_dict={seg_inputs_pl: imgs, d_inputs_pl:img_contours,
                                    K.learning_phase(): 1})
                if not is_running:
                    break
            _, summary = sess.run([train_op_seg, merged_summary_op], 
                             feed_dict={seg_inputs_pl: imgs, d_inputs_pl:img_contours,
                                        K.learning_phase(): 0})
            summary_writer.add_summary(summary, count)
            count += 1
        saver.save(sess, 'RV_Segmentation.ckpt')